In [1]:
import gym
import numpy as np

from gym.wrappers import AtariPreprocessing
gym.new_step_api=True
env = gym.make('Pong-v0')

H = 200 # number of hidden layer neurons
D = 80 * 80 # input dimensionality: 80x80 grid
model = {}
model['W1'] = np.random.randn(H,D) / np.sqrt(D) # "Xavier" initialization
model['W2'] = np.random.randn(H) / np.sqrt(H)
# hyperparameters
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-2
gamma = 0.99 # discount factor for reward
decay_rate = 0.99 # decay factor for RMSProp leaky sum of grad^2
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
  return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

def prepro(I):
  I=np.asarray(I)
  I = I[35:195] # crop
  I = I[::2,::2,0] # downsample by factor of 2
  
  I[I == 144] = 0 # erase background (background type 1)
  
  I[I == 109] = 0 # erase background (background type 2)
  
  I[I != 0] = 1 # everything else (paddles, ball) just set to 1
  return I.astype(float).ravel()

def discount_rewards(r):
  discounted_r = np.zeros_like(r)
  running_add = 0
  for t in reversed(range(0, r.size)):
    if r[t] != 0: running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
    running_add = running_add * gamma + r[t]
    discounted_r[t] = running_add
  return discounted_r

def policy_forward(x):
  h = np.dot(model['W1'], x)
  h[h<0] = 0 # ReLU nonlinearity
  logp = np.dot(model['W2'], h)
  p = sigmoid(logp)
  return p, h # return probability of taking action 2, and hidden state

def policy_backward(epx, eph, epdlogp):
  """ backward pass. (eph is array of intermediate hidden states) """
  dW2 = np.dot(eph.T, epdlogp).ravel()
  dh = np.outer(epdlogp, model['W2'])
  dh[eph <= 0] = 0 # backpro prelu
  dW1 = np.dot(dh.T, epx)
  return {'W1':dW1, 'W2':dW2}

def model_step(model, observation, prev_x):

  cur_x = prepro(observation)
  x = cur_x - prev_x if prev_x is not None else np.zeros(D)
  prev_x = cur_x
  
  # forward the policy network and sample an action from the returned probability
  aprob, _ = policy_forward(x)
  action = 2 if aprob >= 0.5 else 3 # roll the dice!
  
  return action, prev_x

def play_game(env, model):
  observation = env.reset()

  frames = []
  cumulated_reward = 0

  prev_x = None # used in computing the difference frame

  for t in range(1000):
      frames.append(env.render(mode = 'rgb_array'))
      action, prev_x = model_step(model, observation, prev_x)
      observation, reward, done, info = env.step(action)
      cumulated_reward += reward
      if done:
          print("Episode finished after {} timesteps, accumulated reward = {}".format(t+1, cumulated_reward))
          break
  print("Episode finished without success, accumulated reward = {}".format(cumulated_reward))
  env.close()
  display_frames_as_gif(frames)

def train_model(env, model, total_episodes = 100):
  hist = []
  observation = env.reset()

  prev_x = None # used in computing the difference frame
  xs,hs,dlogps,drs = [],[],[],[]
  running_reward = None
  reward_sum = 0
  episode_number = 0

  while True:
  
    cur_x = prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(x)
    action = 2 if np.random.uniform() < aprob else 3 # roll the dice!

    # record various intermediates (needed later for backprop)
    xs.append(x) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 # a "fake label"
    dlogps.append(y - aprob) # grad that encourages the action that was taken to be taken (see http://cs231n.github.io/neural-networks-2/#losses if confused)

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs)
      eph = np.vstack(hs)
      epdlogp = np.vstack(dlogps)
      epr = np.vstack(drs)
      xs,hs,dlogps,drs = [],[],[],[] # reset array memory

      # compute the discounted reward backwards through time
      discounted_epr = discount_rewards(epr)
      # standardize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_epr -= np.mean(discounted_epr)
      discounted_epr /= np.std(discounted_epr)

      epdlogp *= discounted_epr # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(epx, eph, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
      hist.append((episode_number, reward_sum, running_reward))
      print ('episode %f, reward total was %f. running mean: %f' % (episode_number, reward_sum, running_reward))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_x = None
      if episode_number == total_episodes: 
        return hist

   
    

C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\envs\registration.py:592: UserWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.warn(
C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_ap

In [2]:
%time hist1 = train_model(env, model, total_episodes=5500)

C:\Users\owner\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


episode 1.000000, reward total was -21.000000. running mean: -21.000000
episode 2.000000, reward total was -18.000000. running mean: -20.970000
episode 3.000000, reward total was -21.000000. running mean: -20.970300
episode 4.000000, reward total was -20.000000. running mean: -20.960597
episode 5.000000, reward total was -21.000000. running mean: -20.960991
episode 6.000000, reward total was -21.000000. running mean: -20.961381
episode 7.000000, reward total was -21.000000. running mean: -20.961767
episode 8.000000, reward total was -20.000000. running mean: -20.952150
episode 9.000000, reward total was -21.000000. running mean: -20.952628
episode 10.000000, reward total was -21.000000. running mean: -20.953102
episode 11.000000, reward total was -21.000000. running mean: -20.953571
episode 12.000000, reward total was -21.000000. running mean: -20.954035
episode 13.000000, reward total was -21.000000. running mean: -20.954495
episode 14.000000, reward total was -21.000000. running mean

episode 114.000000, reward total was -21.000000. running mean: -20.941842
episode 115.000000, reward total was -21.000000. running mean: -20.942423
episode 116.000000, reward total was -21.000000. running mean: -20.942999
episode 117.000000, reward total was -21.000000. running mean: -20.943569
episode 118.000000, reward total was -21.000000. running mean: -20.944133
episode 119.000000, reward total was -21.000000. running mean: -20.944692
episode 120.000000, reward total was -20.000000. running mean: -20.935245
episode 121.000000, reward total was -19.000000. running mean: -20.915893
episode 122.000000, reward total was -21.000000. running mean: -20.916734
episode 123.000000, reward total was -19.000000. running mean: -20.897567
episode 124.000000, reward total was -20.000000. running mean: -20.888591
episode 125.000000, reward total was -21.000000. running mean: -20.889705
episode 126.000000, reward total was -21.000000. running mean: -20.890808
episode 127.000000, reward total was -

episode 225.000000, reward total was -21.000000. running mean: -20.812880
episode 226.000000, reward total was -21.000000. running mean: -20.814752
episode 227.000000, reward total was -21.000000. running mean: -20.816604
episode 228.000000, reward total was -20.000000. running mean: -20.808438
episode 229.000000, reward total was -21.000000. running mean: -20.810354
episode 230.000000, reward total was -21.000000. running mean: -20.812250
episode 231.000000, reward total was -20.000000. running mean: -20.804128
episode 232.000000, reward total was -21.000000. running mean: -20.806086
episode 233.000000, reward total was -21.000000. running mean: -20.808025
episode 234.000000, reward total was -21.000000. running mean: -20.809945
episode 235.000000, reward total was -18.000000. running mean: -20.781846
episode 236.000000, reward total was -21.000000. running mean: -20.784027
episode 237.000000, reward total was -21.000000. running mean: -20.786187
episode 238.000000, reward total was -

episode 336.000000, reward total was -20.000000. running mean: -20.583329
episode 337.000000, reward total was -20.000000. running mean: -20.577495
episode 338.000000, reward total was -20.000000. running mean: -20.571720
episode 339.000000, reward total was -20.000000. running mean: -20.566003
episode 340.000000, reward total was -21.000000. running mean: -20.570343
episode 341.000000, reward total was -21.000000. running mean: -20.574640
episode 342.000000, reward total was -21.000000. running mean: -20.578893
episode 343.000000, reward total was -21.000000. running mean: -20.583104
episode 344.000000, reward total was -21.000000. running mean: -20.587273
episode 345.000000, reward total was -21.000000. running mean: -20.591401
episode 346.000000, reward total was -21.000000. running mean: -20.595487
episode 347.000000, reward total was -21.000000. running mean: -20.599532
episode 348.000000, reward total was -20.000000. running mean: -20.593536
episode 349.000000, reward total was -

...

episode 4962.000000, reward total was -15.000000. running mean: -17.287657
episode 4963.000000, reward total was -20.000000. running mean: -17.314780
episode 4964.000000, reward total was -19.000000. running mean: -17.331633
episode 4965.000000, reward total was -21.000000. running mean: -17.368316
episode 4966.000000, reward total was -13.000000. running mean: -17.324633
episode 4967.000000, reward total was -17.000000. running mean: -17.321387
episode 4968.000000, reward total was -21.000000. running mean: -17.358173
episode 4969.000000, reward total was -19.000000. running mean: -17.374591
episode 4970.000000, reward total was -20.000000. running mean: -17.400845
episode 4971.000000, reward total was -16.000000. running mean: -17.386837
episode 4972.000000, reward total was -15.000000. running mean: -17.362968
episode 4973.000000, reward total was -13.000000. running mean: -17.319339
episode 4974.000000, reward total was -17.000000. running mean: -17.316145
episode 4975.000000, rewa

episode 5072.000000, reward total was -16.000000. running mean: -17.234943
episode 5073.000000, reward total was -18.000000. running mean: -17.242594
episode 5074.000000, reward total was -18.000000. running mean: -17.250168
episode 5075.000000, reward total was -17.000000. running mean: -17.247666
episode 5076.000000, reward total was -17.000000. running mean: -17.245190
episode 5077.000000, reward total was -15.000000. running mean: -17.222738
episode 5078.000000, reward total was -15.000000. running mean: -17.200510
episode 5079.000000, reward total was -15.000000. running mean: -17.178505
episode 5080.000000, reward total was -15.000000. running mean: -17.156720
episode 5081.000000, reward total was -16.000000. running mean: -17.145153
episode 5082.000000, reward total was -15.000000. running mean: -17.123701
episode 5083.000000, reward total was -17.000000. running mean: -17.122464
episode 5084.000000, reward total was -15.000000. running mean: -17.101240
episode 5085.000000, rewa

episode 5182.000000, reward total was -14.000000. running mean: -17.279387
episode 5183.000000, reward total was -15.000000. running mean: -17.256594
episode 5184.000000, reward total was -17.000000. running mean: -17.254028
episode 5185.000000, reward total was -11.000000. running mean: -17.191487
episode 5186.000000, reward total was -15.000000. running mean: -17.169572
episode 5187.000000, reward total was -18.000000. running mean: -17.177877
episode 5188.000000, reward total was -13.000000. running mean: -17.136098
episode 5189.000000, reward total was -14.000000. running mean: -17.104737
episode 5190.000000, reward total was -14.000000. running mean: -17.073690
episode 5191.000000, reward total was -17.000000. running mean: -17.072953
episode 5192.000000, reward total was -20.000000. running mean: -17.102223
episode 5193.000000, reward total was -15.000000. running mean: -17.081201
episode 5194.000000, reward total was -19.000000. running mean: -17.100389
episode 5195.000000, rewa

episode 5292.000000, reward total was -15.000000. running mean: -16.894423
episode 5293.000000, reward total was -16.000000. running mean: -16.885479
episode 5294.000000, reward total was -16.000000. running mean: -16.876624
episode 5295.000000, reward total was -13.000000. running mean: -16.837858
episode 5296.000000, reward total was -17.000000. running mean: -16.839479
episode 5297.000000, reward total was -17.000000. running mean: -16.841084
episode 5298.000000, reward total was -16.000000. running mean: -16.832673
episode 5299.000000, reward total was -18.000000. running mean: -16.844347
episode 5300.000000, reward total was -20.000000. running mean: -16.875903
episode 5301.000000, reward total was -15.000000. running mean: -16.857144
episode 5302.000000, reward total was -15.000000. running mean: -16.838573
episode 5303.000000, reward total was -20.000000. running mean: -16.870187
episode 5304.000000, reward total was -17.000000. running mean: -16.871485
episode 5305.000000, rewa

episode 5402.000000, reward total was -18.000000. running mean: -16.532200
episode 5403.000000, reward total was -18.000000. running mean: -16.546878
episode 5404.000000, reward total was -19.000000. running mean: -16.571410
episode 5405.000000, reward total was -20.000000. running mean: -16.605696
episode 5406.000000, reward total was -14.000000. running mean: -16.579639
episode 5407.000000, reward total was -15.000000. running mean: -16.563842
episode 5408.000000, reward total was -15.000000. running mean: -16.548204
episode 5409.000000, reward total was -18.000000. running mean: -16.562722
episode 5410.000000, reward total was -20.000000. running mean: -16.597095
episode 5411.000000, reward total was -21.000000. running mean: -16.641124
episode 5412.000000, reward total was -17.000000. running mean: -16.644712
episode 5413.000000, reward total was -17.000000. running mean: -16.648265
episode 5414.000000, reward total was -20.000000. running mean: -16.681783
episode 5415.000000, rewa